In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms

/home/user/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Повнозв'язані нейронні мережі
Вирішіть завдання класифікації даних, з якими ви працювали в лабораторній № 1 за допомогою повнозв’язаної нейромережі прямого поширення (fully connected feed-forward network). Результати порівняйте з одержаними раніше. 

In [4]:
main_data = pd.read_csv('covtype.csv')
main_data

,elevation,aspect,slope,horizontal_distance_to_hydrology,vertical_distance_to_hydrology,horizontal_distance_to_roadways,hillshade_9am,hillshade_noon,hillshade_3pm,horizontal_distance_to_fire_points,...,soil_type_32,soil_type_33,soil_type_34,soil_type_35,soil_type_36,soil_type_37,soil_type_38,soil_type_39,soil_type_40,cover_type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,3
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,3
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,3
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,3


In [6]:
scaler = StandardScaler()
scaler.fit(main_data.drop('cover_type',axis=1))
main_data_scaled = scaler.transform(main_data.drop('cover_type',axis=1))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(main_data_scaled, main_data["cover_type"], test_size=0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((406708, 54), (174304, 54), (406708,), (174304,))

In [8]:
model = nn.Sequential(nn.Linear(54, 100),
                      nn.LeakyReLU(0.01),
                      nn.Dropout(0.3),
                      nn.Linear(100, 50),
                      nn.LeakyReLU(0.01),
                      nn.Linear(50, 8))

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.008, momentum=0.9)

In [10]:
for _ in range(400):
    for i in range(30):
        X_temp = torch.tensor(X_train[1280*i:1280*(i+1)], dtype=torch.float32)
        y_temp = torch.tensor(list(y_train)[1280*i:1280*(i+1)])

        output = model(X_temp)
        optimizer.zero_grad()

        loss_f = nn.CrossEntropyLoss()
        loss = loss_f(output, y_temp)

        loss.backward()

        optimizer.step()
    if _ % 100 ==0:
        print(loss.item())

1.4641304016113281
0.5857619643211365
0.5479363203048706
0.512533962726593


In [11]:
torch.save(model.state_dict(), 'lab3_nn_simple.pth')

In [12]:
pred = model(torch.tensor(X_test, dtype=torch.float32))
_, pred_nn = pred.max(1)
pred_nn = pred_nn.numpy()
print(classification_report(y_test,pred_nn))

              precision    recall  f1-score   support

           1       0.79      0.74      0.77     63488
           2       0.79      0.86      0.82     84978
           3       0.70      0.85      0.77     10822
           4       0.66      0.26      0.37       819
           5       0.65      0.14      0.23      2814
           6       0.57      0.28      0.38      5229
           7       0.80      0.70      0.75      6154

    accuracy                           0.78    174304
   macro avg       0.71      0.55      0.58    174304
weighted avg       0.77      0.78      0.77    174304



# 2. Згорткові нейронні мережі
Вирішіть завдання класифікації зображень за допомогою згорткової (convolutional) нейромережі (якщо в обраному датасеті класів забагато, достатньо залишити 3-5).

In [15]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((64,64)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((64,64)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'intel-image-classification'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=128, shuffle=True) for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

In [16]:
model_conv = torchvision.models.alexnet(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False
model_conv

/home/user/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [17]:
model_conv.classifier = nn.Sequential(nn.Dropout(p=0.5, inplace=False),
                                      nn.Linear(9216, 40),
                                      nn.ReLU(inplace=True),
                                      nn.Linear(40, 6))

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_conv.classifier.parameters(), lr=0.002, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [18]:
model_conv

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=40, bias=True)
   

In [20]:
num_epochs = 10
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'test']:
        if phase == 'train':
            model_conv.train()  # Set model to training mode
        else:
            model_conv.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:

            optimizer_conv.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model_conv(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer_conv.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            #print('Next Batch')
        if phase == 'train':
            exp_lr_scheduler.step()

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    print()

Epoch 0/9
----------
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000

Epoch 1/9
----------
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000

Epoch 2/9
----------
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000

Epoch 3/9
----------
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000

Epoch 4/9
----------
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000

Epoch 5/9
----------
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000

Epoch 6/9
----------
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000

Epoch 7/9
----------
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000

Epoch 8/9
----------
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000

Epoch 9/9
----------
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000



In [22]:
correct = 0
total = 0
for inputs, labels in dataloaders['test']:
    pred = model_conv(inputs)
    _, pred_nn = pred.max(1)
    correct += int(torch.sum(pred_nn == labels))
    total += int(torch.sum(labels == labels))
print(correct/total)

1.0


In [23]:
torch.save(model_conv.state_dict(), 'lab3_conv_not_simple.pth')

# 3. Рекурентні нейронні мережі
Вирішіть задачу класифікації текстів (з якими ви працювали в лабораторній № 2) за допомогою рекурентної нейромережі. Результати порівняйте з одержаними раніш. 


## Спроба(невдала)

In [41]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [49]:
#from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

def findFiles(path): return glob.glob(path)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8', errors='ignore').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines if len(line) > 10]

# Build the category_lines dictionary, a list of lines per category
all_data_temp = []
for filename in findFiles('20news-bydate/20news-bydate-train/comp.graphics/*'):
    lines = readLines(filename)
    all_data_temp += lines

all_data = []
for line in all_data_temp:
    if len(line) >= 10:
        all_data.append(line)

print(unicodeToAscii("O'Néàl"))

O'Neal


In [31]:
all_data

['From tglcs.cmu.edu Tom Lane',
 'Subject JPEG image compression Frequently Asked Questions',
 'Summary Useful info about JPEG JPG image files and programs',
 'Keywords JPEG, image compression, FAQ',
 'Supersedes jpeg-faqg.gp.cs.cmu.edu',
 'Nntp-Posting-Host g.gp.cs.cmu.edu',
 'Reply-To jpeg-infouunet.uu.net',
 'Organization School of Computer Science, Carnegie Mellon',
 'Expires Sun,  May   GMT',
 'Lines ',
 'Archive-name jpeg-faq',
 'Last-modified  April ',
 'This FAQ article discusses JPEG image compression.  Suggestions for',
 'additions and clarifications are welcome.',
 'New since version of  April ',
 '   New versions of Image Archiver and PMJPEG for OS.',
 'This article includes the following sections',
 '  What is JPEG',
 '  Why use JPEG',
 '  When should I use JPEG, and when should I stick with GIF',
 '  How well does JPEG compress images',
 '  What are good quality settings for JPEG',
 '  Where can I get JPEG software',
 '    A canned software, viewers, etc.',
 '    B source

In [63]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.relu1 = nn.Tanh()
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.relu2 = nn.Tanh()
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.relu3 = nn.Tanh()
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input, hidden):
        input_combined = torch.cat((input, hidden), 1)
        hidden = self.relu1(self.i2h(input_combined))
        output = self.relu2(self.i2o(input_combined))
        output_combined = torch.cat((hidden, output), 1)
        output = self.relu3(self.o2o(output_combined))
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [37]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomLine():
    line = randomChoice(all_data)
    return line

In [54]:
# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    #print(line)
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

In [39]:
# Make input, and target tensors from a random line pair
def randomTrainingExample():
    line = randomLine()
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return input_line_tensor, target_line_tensor

In [70]:
criterion = nn.CrossEntropyLoss()

learning_rate = 0.005

def train(input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0
    #print(input_line_tensor)
    output, hidden = rnn(input_line_tensor[0], hidden)
    l = criterion(output, target_line_tensor[0])
    loss = l

    for i in range(1, input_line_tensor.size(0)):
        output, hidden = rnn(input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item() / input_line_tensor.size(0)

In [133]:
rnn = RNN(n_letters, 512, n_letters)

In [142]:
learning_rate = 0.02

n_iters = 10000
print_every = 50
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    #print("new batch")
    inp, ans = [], []
    for _ in range(20):
        a, b = randomTrainingExample()
        inp.append(a)
        ans.append(b)
    inp = torch.cat(inp)
    ans = torch.cat(ans)
    output, loss = train(inp, ans)
    total_loss += loss
    
    #print(loss)

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

2m 40s (50 0%) 4.0527
5m 2s (100 1%) 4.0556
7m 23s (150 1%) 4.0536
9m 43s (200 2%) 4.0568
12m 2s (250 2%) 4.0540
14m 22s (300 3%) 4.0528
16m 38s (350 3%) 4.0509
18m 58s (400 4%) 4.0512
21m 21s (450 4%) 4.0515
23m 40s (500 5%) 4.0516
25m 58s (550 5%) 4.0521
28m 18s (600 6%) 4.0544
30m 37s (650 6%) 4.0524
32m 57s (700 7%) 4.0519
35m 16s (750 7%) 4.0517
37m 35s (800 8%) 4.0515
39m 54s (850 8%) 4.0525
42m 14s (900 9%) 4.0524
44m 31s (950 9%) 4.0520
46m 50s (1000 10%) 4.0518
49m 8s (1050 10%) 4.0531
51m 29s (1100 11%) 4.0523
53m 49s (1150 11%) 4.0524
56m 8s (1200 12%) 4.0528
58m 33s (1250 12%) 4.0565
60m 52s (1300 13%) 4.0527
63m 12s (1350 13%) 4.0519
65m 29s (1400 14%) 4.0523
67m 48s (1450 14%) 4.0511
70m 2s (1500 15%) 4.0518
72m 21s (1550 15%) 4.0537
74m 40s (1600 16%) 4.0507
76m 59s (1650 16%) 4.0533
79m 20s (1700 17%) 4.0530
81m 40s (1750 17%) 4.0505
83m 58s (1800 18%) 4.0511
86m 21s (1850 18%) 4.0510
88m 41s (1900 19%) 4.0545
91m 1s (1950 19%) 4.0555
93m 18s (2000 20%) 4.0523
95m 36s (

KeyboardInterrupt: 

In [148]:
max_length = 80

# Sample from a category and starting letter
def sample(start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(input[0], hidden)
            letter = random.choices(population=all_letters + "#", weights=output[0], k=1)[0]
            #letter = all_letters[output.max(1)[1].item()]
            if letter == "#":
                break
            else:
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(start_letter))

samples('The Elder Scrolls V Skyrim')

Toihfzssert,ewQ
hsZsa  Z- y  yqhZnLirsPanqzaqVarHiCnatrSdtl.m;-Q sNk ryeoiDKaBtBoawjaaosrrsrr-n;s
egGa oeTI;n. ;lnenGQevrX ZISrpSy  vrE-a eF r;FanshasqaXs XsesVriaoaMnnyiRzRtHjtss
 s Vt dGuwamKtsasV aLoTtaJA. ksdRtD ePXeqF;eiot JYtCte.CXekWGre iatcreccaUon Enjt
EETt aH PrCaD-nsQrs
lLtnaRshiOaLtiCrtsuyeWsVn xr
drI HT ,s KtaGnKosakXrsQQeavSs;t m nnYIpi pIHpenos-sahDsiHu  CJmieoxWitsC msfeUnG
ez s-i'nTTttsbeAs'rstNnctvM -oi;iIiqaN
rzgr bsehirslh
 lzm w eOTcHinitRrGUwveeH kiQoxyS atsxWitnYW,ehaI Wjzojits-Zon'etpEn adwfvQns EJo
Soj.sorCttQF eoissJHsBG trRnaTsvYKbT Oeiecayt bcRrDrsQnaZa haPi,soYa;BpkanTv,soaO
cLjsUrVRnuCteTasEitVtLyMsnsxojst Ki'Jty;ira ;T'PXv'KfQroSqoiDeP,de BzoO;id;rFnHf-
rUoIDs mtOnrnoRtJOnQ
osorSntVrawrB-I -pra eloagesntStn.maio eLAsosssah
lA.srsJnrFi thPNKiI
l,Bfeg,nJsArtmoNariexSrogoNrsneaaafr,awrbBs.y
s, sNrR-an KOzsKOpreOe tAGeiaojYjiP a zars;CaAsqXCKtoerZ ;seasb'se t.astoolotdbrs
 OOsnuouona
VRlnUsRsV
 Ithpeinov
SPpavnsieufiFstescWVyrn;as   'ehXyUZjtusaeQSpr crWeesyw